In [1]:
import numpy as np
import pandas as pd
import pyodbc
import csv
import pyworms

# DEBUG
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from warnings import filterwarnings
filterwarnings("ignore", category=UserWarning, message='.*')

# Output file path
outdir = "D:\\00-GCOOS\\00-MBON\\CAGES\\MS\\data\\merged\\"

# THE ORIGINAL CAGES DATABASE FILE is served via WAF but cannot be queried from there
# -> download to local HD
#db_file = 'https://gcoos4.geos.tamu.edu/WAF/MBON/CAGES/CAGES.accdb'
db_file = "D:\\00-GCOOS\\00-MBON\\CAGES_ORIG\CAGES.accdb"
user = 'user'
password = 'pw'
# open the database connection:
cnxn = pyodbc.connect('DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={};Uid={};Pwd={};'.format(db_file, user, password))

### GET DATA TABLES

# -- TRAWLS and SPECIES
query = 'SELECT * FROM "Mississippi Trawls" AS trawls LEFT JOIN "Mississippi Species" AS species ON \
trawls.[Species Code] = species.[Species Code]'
df = pd.read_sql(query, cnxn)
# Remove duplicate columns
df = df.loc[:, ~df.columns.duplicated(keep='first')]
# Convert Species Code back to integer
intcols= ['Species Code', 'Sample Code', 'Measured']
for acol in intcols:
    df[acol] = df[acol].astype('Int64')

# -- LENGTHS
query = 'SELECT * FROM "Mississippi Lengths"'
df_add = pd.read_sql(query, cnxn)
# Convert Species Code back to integer
intcols= ['Species Code', 'Sample Code']
for acol in intcols:
    df_add[acol] = df_add[acol].astype('Int64')
df = pd.merge(df, df_add, on=['Sample Code','Species Code'],how='left')

# -- CPUE
query = 'SELECT * FROM "Mississippi CPUE"'
df_add = pd.read_sql(query, cnxn)
df_add['Species Code'] = df_add['Species Code'].astype('Int64')
df = pd.merge(df_add, df, on=['Sample Code', 'Species Code'],how='left')

# -- STATIONS
query = 'SELECT * FROM "Mississippi Stations"'
df_add = pd.read_sql(query, cnxn)
df = pd.merge(df, df_add, on=['Station Code'],how='left')

# -- HYDROLOGICAL
query = 'SELECT * FROM "Mississippi Hydrological"'
df_add = pd.read_sql(query, cnxn)
df = pd.merge(df, df_add, on=['Sample Code'],how='left')

# Close the database connection
cnxn.close()

# Generate Datetime field
df['datestr'] = df['YYYY'].astype(str) + '-' + df['MM'].astype(str) + '-' + df['DD'].astype(str) + ' 12:00'
df['Datetime'] = pd.to_datetime(df['datestr'], utc=True)

# drop columns that are not needed for output
df.drop(['datestr', 'YYYY', 'MM', 'DD'], axis=1, inplace=True)

# sort by date
df.sort_values(['Datetime'], axis=0, ascending=True, inplace=True, ignore_index=True)
# Format datetime string
df['Datetime'] = df['Datetime'].dt.strftime('%Y-%m-%dT%H:%MZ')

# Get rid of spaces in column names
for acol in df.columns.to_list():
    df.rename({acol: acol.replace(" ","")}, axis="columns", inplace=True)

cols_out = [ 'Datetime','Latitude','Longitude','SampleCode','StationCode','Description','Bay','SpeciesCode','ScientificName',
            'Phylum','Class','Order','Family','cpue','Measured','TotalNumber','TotalWeight','Length','Weight','Depth',
            'SurfaceSalinity','BottomSalinity','AverageSalinity','SurfaceTemperature','BottomTemperature','AverageTemperature',
            'SurfaceDO','BottomDO','AverageDO']

### OUTPUT TO FILE
# Write merged data out to a .csv file
df[cols_out].to_csv(outdir + "CAGES_CPUE_MS.csv", encoding='utf-8', index=False)

In [2]:
### Generate ERDDAP datasets .XML -snippet using templates according to variable type

## Paths 
# ouput
xml_output = "D:\\00-GCOOS\\00-MBON\\CAGES\\MS\\data\\debug\\" + "CAGES_MS_XML.txt"
# template files
xml_header_temp = "D:\\00-GCOOS\\00-MBON\\CAGES\\erdxml\\cages_erddap_xml\\" + "cages_header_template_MS.txt"
xml_int_temp = "D:\\00-GCOOS\\00-MBON\\CAGES\\erdxml\\cages_erddap_xml\\" + "int_xml_template.txt"
xml_float_temp = "D:\\00-GCOOS\\00-MBON\\CAGES\\erdxml\\cages_erddap_xml\\" + "float_xml_template.txt"
xml_string_temp = "D:\\00-GCOOS\\00-MBON\\CAGES\\erdxml\\cages_erddap_xml\\" + "string_xml_template.txt"

# get types
dt = df.dtypes.to_dict()
int_cols = []
float_cols = []
string_cols = []

# list types
for akey in dt.keys():
     if 'nt64' in str(dt[akey]):
        int_cols.append(akey)
     elif 'float' in str(dt[akey]):
         float_cols.append(akey)
     elif 'obj' in str(dt[akey]) or 'str' in str(dt[akey]):
         string_cols.append(akey)

# These belong to the header snippet (no need to generate separately):
header_vars = ['Datetime', 'Latitude', 'Longitude']

# START concatenating the tamplate snippets together
# --

# 1st, write the header to the output file
with open(xml_output, "w") as output_file:
    with open(xml_header_temp, "r") as file:
        output_file.write(file.read())

# Loop throgh the rest
for acol in cols_out:
    isPhysicalMeasurement = False
    if acol in header_vars:
        continue
    elif acol in int_cols:
        template_file = xml_int_temp
    elif acol in float_cols:
        template_file = xml_float_temp
        isPhysicalMeasurement = True
    else:
        template_file = xml_string_temp
    
    # Open the xml template file for reading
    with open(template_file, 'r') as tempfile:
        # Read the contents of the file
        contents = tempfile.read()
        # Modify the contents as needed
        mod_contents = contents.replace('_VARNAME_', acol)
        
        # Try to figure out the units for some
        if isPhysicalMeasurement:
            if 'temperature' in acol.lower():
                mod_contents = mod_contents.replace('_UNITS_', 'degree_Celcius')
                mod_contents = mod_contents.replace('_CATEGORY_', 'Temperature')
                if 'air' in acol.lower():
                    mod_contents = mod_contents.replace('_DESTNAME_', 'air_temperature')
                elif 'surface' in acol.lower():
                    mod_contents = mod_contents.replace('_DESTNAME_', 'sea_surface_temperature')
                elif 'bottom' in acol.lower():
                    mod_contents = mod_contents.replace('_DESTNAME_', 'sea_water_temperature_at_sea_floor')
                elif 'average' in acol.lower():
                    mod_contents = mod_contents.replace('_DESTNAME_', 'sea_water_temperature')
            elif 'turbidity' in acol.lower():
                mod_contents = mod_contents.replace('_UNITS_', 'NTU')
                mod_contents = mod_contents.replace('_CATEGORY_', 'Turbidity')
                mod_contents = mod_contents.replace('_DESTNAME_', 'sea_water_turbidity')
            elif 'salinity' in acol.lower():
                mod_contents = mod_contents.replace('_UNITS_', 'PSU')
                mod_contents = mod_contents.replace('_CATEGORY_', 'Salinity')
                if 'surface' in acol.lower():
                    mod_contents = mod_contents.replace('_DESTNAME_', 'sea_surface_salinity')
                elif 'bottom' in acol.lower():
                    mod_contents = mod_contents.replace('_DESTNAME_', 'sea_water_salinity_at_sea_floor')
                elif 'average' in acol.lower():
                    mod_contents = mod_contents.replace('_DESTNAME_', 'sea_water_salinity')
            elif 'DO' in acol or 'dissolved' in acol.lower() or 'oxygen' in acol.lower():
                mod_contents = mod_contents.replace('_UNITS_', 'mg l-1')
                mod_contents = mod_contents.replace('_CATEGORY_', 'Dissolved O2')
                mod_contents = mod_contents.replace('_DESTNAME_', acol)
            else:
                mod_contents = mod_contents.replace('_DESTNAME_',acol)

    # Open the ouput file for appending
    with open(xml_output, 'a') as file2:
        # Write the modified contents back to the file
        file2.write(mod_contents)

### Add the closing tag to the output xml file:
with open(xml_output, 'a') as output_file:
    output_file.write("</dataset>")

In [4]:
float_cols

['cpue',
 'TotalWeight',
 'Length',
 'Weight',
 'Latitude',
 'Longitude',
 'Depth',
 'SurfaceSalinity',
 'BottomSalinity',
 'AverageSalinity',
 'SurfaceTemperature',
 'BottomTemperature',
 'AverageTemperature',
 'SurfaceDO',
 'BottomDO',
 'AverageDO']

In [3]:
int_cols

['SampleCode', 'StationCode', 'SpeciesCode', 'Measured', 'TotalNumber']